In [17]:
import requests
import json
import pandas as pd
import urllib.parse
from pathlib import Path
from datetime import date
from datetime import timedelta

# GLOBAL VARIABLES #
AFF_SYNC_COLUMNS = ['affiliateid','affiliateein','affiliatepercapitapin','IsChartered','CharterDate','electionmonth','officertermstartmonth','iselectionyearodd','electiontermyears','updatedby','updatedat']
APC_SYNC_COLUMNS = ['AffiliatePerCapitaId','FiscalYearEndMonth','FiscalYearEndDay','PayPerCapitaToAFT','InvoicedByAFT','IncludeAFLCIOPerCapita','AFLCIOAmount','AffiliateBillingFrequencyId','HasOccupationalLiabilityInsurance','FiduciaryBondCoverageID','AccidentInsuranceUnits','ConventionDelegationEligibility','IsAgencyFee','IsStateDues','DeliveryType','GroupNumber','UpdatedBy','UpdatedAt','DeletedAt']
ADIV_SYNC_COLUMNS = ['Action','AffiliateDivisionId','Affiliate','Division','CreatedBy','CreatedAt','UpdatedBy','UpdatedAt','DeletedAt']

YESTERDAY_DATE = (date.today() - timedelta(days = 1)).strftime("%m/%d/%Y")
DATE_TO_RUN = YESTERDAY_DATE

TABLES = ['Affiliate','Division', 'AffiliateDivision', 'AffiliatePerCapita','Accounts','StateFederation','AffiliateType','AffiliateDesignation','AffiliateGeoReach']

# JSON PRINT HELPER #
def jprint(output):
    print(json.dumps(output, indent=4))

class KnackAFT:
    def __init__(self):
        # API #
        self.API_KEY = '1b8065b3-d5a5-4586-946a-d9f5d315963f'
        self.APP_ID = '6157aca138a38604ae371cd9'
        
        # HTTP REQUESTS #
        self.GET_HEADERS = {'X-Knack-REST-API-KEY':self.API_KEY,'X-Knack-Application-Id':self.APP_ID}
        self.POST_HEADERS = {'X-Knack-REST-API-KEY':self.API_KEY,'X-Knack-Application-Id':self.APP_ID,'content-type':'application/json'}
        self.API_URL = f'https://api.knack.aft.org/v1/'
        self.LOADER_URL = f'https://loader.knack.aft.org/v1/applications/{self.APP_ID}'

        # INTERNAL #
        self.APP_DICT = {}

        res = requests.get(url=self.LOADER_URL)
        objects = res.json()['application']['objects']

        for obj in objects:
            fields = {}
            name = obj['name']
            key = obj['key']
            
            if name in TABLES:
                for item in obj['fields']:
                    fields.update({item['name']:item['key']})
                self.APP_DICT.update({name:{'obj_id':key,'fields':fields}})
        
    # function to return key for any value
    def get_key(self, dictionary ,val):
        for key, value in dictionary.items():
            if val == value:
                return key

        return ''
        
    
    # GET and format json from requestURL
    def getJSON(self, url):
        r = requests.get(url = self.API_URL + url, headers = self.GET_HEADERS)
        return r.json()
    
    def getObjectJSON(self, object_name):
        return (self.getJSON('objects/' + self.APP_DICT[object_name]['obj_id']))['object']
        
    def find_matches(self, object_name, field_name, match_val):
        field_id = self.APP_DICT[object_name]['fields'][field_name]
        object_id = self.APP_DICT[object_name]['obj_id']
        
        match_filter = {'match':'and', 'rules':[{'field':field_id, 'operator':'is', 'value': match_val}]}
        filter_for_url = urllib.parse.quote(json.dumps(match_filter))
        request_url = "objects/" + object_id + "/records?filters=" + filter_for_url
        res = self.getJSON(request_url)
        if res["total_records"] == 0:
            return ''
        else:
            return res["records"]
        
    def find_records_updated_at_date(self, knack_object, date):
        #Convert to IDs
        knack_object_id = self.APP_DICT[knack_object]['obj_id']
        field_to_match_id = self.APP_DICT[knack_object]['fields']['mdate']
        aftdbupdate_id = self.APP_DICT[knack_object]['fields']['AFTDBUpdate']

        #Get Id {'field':field_to_match_id, 'operator':'is', 'value':date}, 
        match_filter = {'match':'and', 'rules':[{'field':aftdbupdate_id, 'operator':'is', 'value':0}]}
        filter_for_url = urllib.parse.quote(json.dumps(match_filter))
        request_url = "https://api.knack.aft.org/v1/objects/" + knack_object_id + "/records?filters=" + filter_for_url

        r = requests.get(url = request_url, headers = self.GET_HEADERS)
        #print(json.dumps(r.json(), indent=4))
        res_json_dict = json.loads(json.dumps(r.json()))
        if res_json_dict["total_records"] == 0:
            return ''
        else:
            return res_json_dict["records"]
        
    def get_userid(self, user):
        #Convert to IDs
        knack_object_id = self.APP_DICT['Accounts']['obj_id']
        knack_field_id = self.APP_DICT['Accounts']['fields']['KnackUserID']

        request_url = "https://api.knack.aft.org/v1/objects/" + knack_object_id + "/records/" +  user
        r = requests.get(url = request_url, headers = self.GET_HEADERS)
        #print(json.dumps(r.json(), indent=4))
        res_json_dict = json.loads(json.dumps(r.json()))
        return res_json_dict[knack_field_id]
    
    def get_connection(self,connection_name,field,connection):
        #Convert to IDs
        knack_object_id = self.APP_DICT[connection_name]['obj_id']
        knack_field_id = self.APP_DICT[connection_name]['fields'][field]

        request_url = "https://api.knack.aft.org/v1/objects/" + knack_object_id + "/records/" +  connection
        r = requests.get(url = request_url, headers = self.GET_HEADERS)
        res_json_dict = json.loads(json.dumps(r.json()))

        if 'total_records' in res_json_dict:
            if res_json_dict["total_records"] == 0:
                return ''
            else:
                return res_json_dict[knack_field_id]
        else:
            return res_json_dict[knack_field_id]



client = KnackAFT()
aff_fields = {}
apc_fields = {}
adiv_fields = {}

for name in AFF_SYNC_COLUMNS:
    aff_fields.update({name:client.APP_DICT['Affiliate']['fields'][name]})
        
for name in APC_SYNC_COLUMNS:
    apc_fields.update({name:client.APP_DICT['AffiliatePerCapita']['fields'][name]})
    
for name in ADIV_SYNC_COLUMNS:
    adiv_fields.update({name:client.APP_DICT['AffiliateDivision']['fields'][name]})


def affiliate_runner():
    output_df = []
    records = client.find_records_updated_at_date('Affiliate', DATE_TO_RUN)
    if records:
        for record in records:
            output = {}
            for k,v in aff_fields.items():
                output.update({k:str(record[v])})
                
            user_conn = record[client.APP_DICT['Affiliate']['fields']['muser']+'_raw'][0]['id']
            mdate = record[client.APP_DICT['Affiliate']['fields']['mdate']]
            muser = client.get_userid(user_conn)
            
            pid = record[client.APP_DICT['Affiliate']['fields']['ParentAffiliateID']+'_raw']
            if pid:
                val = client.get_connection('StateFederation','AffiliateId',record[client.APP_DICT['Affiliate']['fields']['ParentAffiliateID']+'_raw'][0]['id'])
                output.update({'ParentAffiliateID':val})
            else:
                output.update({'ParentAffiliateID':''})
        
            val = client.get_connection('AffiliateType','AffiliateTypeId',record[client.APP_DICT['Affiliate']['fields']['AffiliateTypeID']+'_raw'][0]['id'])
            output.update({'AffiliateTypeID':val})
            
            val = client.get_connection('AffiliateDesignation','AffiliateDesignationId',record[client.APP_DICT['Affiliate']['fields']['AffiliateDesignationID']+'_raw'][0]['id'])
            output.update({'AffiliateDesignationID':val})
            
            val = client.get_connection('AffiliateGeoReach','AffiliateGeoReachId',record[client.APP_DICT['Affiliate']['fields']['AffiliateGeoReachID']+'_raw'][0]['id'])
            output.update({'AffiliateGeoReachID':val})
            
            output.update({'updatedat':str(mdate)})
            output.update({'updatedby':str(muser)})
            output_df.append(output)

    jprint(output_df)
    #return pd.DataFrame(output_df, dtype=str)

def affiliatepercapita_runner():
    output_df = []
    records = client.find_records_updated_at_date('AffiliatePerCapita', DATE_TO_RUN)
    if records:
        for record in records:
            output = {}
            for k,v in apc_fields.items():
                output.update({k:str(record[v])})
                
            user_conn = record[client.APP_DICT['AffiliatePerCapita']['fields']['muser']+'_raw'][0]['id']
            mdate = record[client.APP_DICT['AffiliatePerCapita']['fields']['mdate']]
            muser = client.get_userid(user_conn)
            
            output.update({'UpdatedAt':str(mdate)})
            output.update({'UpdatedBy':str(muser)})
            output_df.append(output)
    jprint(output_df)        
    #return pd.DataFrame(output_df, dtype=str)

def affiliatedivision_runner():
    output_df = []
    records = client.find_records_updated_at_date('AffiliateDivision', DATE_TO_RUN)
    if records:
        for record in records:
            output = {}
            for k,v in adiv_fields.items():
                output.update({k:str(record[v])})
                
            user_conn = record[client.APP_DICT['AffiliateDivision']['fields']['muser']+'_raw'][0]['id']
            mdate = record[client.APP_DICT['AffiliateDivision']['fields']['mdate']]
            muser = client.get_userid(user_conn)
            
            val = client.get_connection('Affiliate','affiliateid',record[client.APP_DICT['AffiliateDivision']['fields']['Affiliate']+'_raw'][0]['id'])
            output.update({'Affiliate':val})

            val = client.get_connection('Division','DivisionId',record[client.APP_DICT['AffiliateDivision']['fields']['Division']+'_raw'][0]['id'])
            output.update({'Division':val})
            
            output.update({'UpdatedAt':str(mdate)})
            output.update({'UpdatedBy':str(muser)})
            out_sort = {}
            for k,v in output.items():
                if k == 'Affiliate':
                    out_sort.update({'AffiliateID':v})
                elif k == 'Division':
                    out_sort.update({'DivisionID':v})
                else:
                    out_sort.update({k:v})
            if out_sort['Action'] == 'Delete':
                if out_sort['AffiliateDivisionId']:
                    output_df.append(out_sort)
            else:
                output_df.append(out_sort)
    jprint(output_df)        
    #return pd.DataFrame(output_df, dtype=str)

#jprint(client.APP_DICT)
    
affiliate_runner()
affiliatepercapita_runner()
affiliatedivision_runner()

[
    {
        "affiliateid": "10535",
        "affiliateein": "",
        "affiliatepercapitapin": "",
        "IsChartered": "0",
        "CharterDate": "",
        "electionmonth": "",
        "officertermstartmonth": "",
        "iselectionyearodd": "",
        "electiontermyears": "",
        "updatedby": "19",
        "updatedat": "10/25/2022",
        "ParentAffiliateID": 56,
        "AffiliateTypeID": 4,
        "AffiliateDesignationID": 1,
        "AffiliateGeoReachID": 2
    },
    {
        "affiliateid": "3914",
        "affiliateein": "873510384",
        "affiliatepercapitapin": "351734",
        "IsChartered": "1",
        "CharterDate": "02/22/2022",
        "electionmonth": "",
        "officertermstartmonth": "",
        "iselectionyearodd": "",
        "electiontermyears": "",
        "updatedby": "19",
        "updatedat": "01/11/2023",
        "ParentAffiliateID": 8,
        "AffiliateTypeID": 4,
        "AffiliateDesignationID": 2,
        "AffiliateGeoReachID": 

In [10]:
#---------------------------------------------------------------------
# THIS SCRIPT IS MEANT TO FAIL EVERYDAY EXCEPT THE 15th OF THE MONTH
#   This is done to prevent the downstream workflow for beneficiary knack
#   from running till the 15th when it is needed.
#---------------------------------------------------------------------
import sys
from datetime import date
from datetime import timedelta

#Get today's date and extarcts the day
today_day = date.today().day

#If today is the 15th of the month, print and exit successfully.
#If today is not the 15th, print error and exit with error.
if today_day == 15:
    print(f'Today is the {today_day} of the month, running workflow...')
else:
    print('Today is not the 15th of the month, failing workflow.')
    sys.exit('Date not valid')

Today is the 15 of the month, running workflow...


In [15]:

import requests
import json
import pandas as pd
import urllib.parse
from pathlib import Path
from datetime import date
from datetime import timedelta

# GLOBAL VARIABLES #
AFF_SYNC_COLUMNS = ['affiliateid','affiliateein','affiliatepercapitapin','IsChartered','CharterDate','electionmonth','officertermstartmonth','iselectionyearodd','electiontermyears','updatedby','updatedat']
APC_SYNC_COLUMNS = ['AffiliatePerCapitaId','FiscalYearEndMonth','FiscalYearEndDay','PayPerCapitaToAFT','InvoicedByAFT','IncludeAFLCIOPerCapita','AFLCIOAmount','AffiliateBillingFrequencyId','HasOccupationalLiabilityInsurance','FiduciaryBondCoverage','AccidentInsuranceUnits','ConventionDelegationEligibility','IsAgencyFee','IsStateDues','DeliveryType','GroupNumber','UpdatedBy','UpdatedAt','DeletedAt']
TABLES = ['Affiliate','AffiliatePerCapita','AffiliateDivision','Division','Accounts','StateFederation','AffiliateType','FiduciaryBondCoverage','AffiliateDesignation','AffiliateInactiveReason','AffiliateGeoReach']

# JSON PRINT HELPER #
def jprint(output):
    print(json.dumps(output, indent=4))

class KnackAFT:
    def __init__(self):
        # API #
        self.API_KEY = '1b8065b3-d5a5-4586-946a-d9f5d315963f'
        self.APP_ID = '6157aca138a38604ae371cd9'
        
        # HTTP REQUESTS #
        self.GET_HEADERS = {'X-Knack-REST-API-KEY':self.API_KEY,'X-Knack-Application-Id':self.APP_ID}
        self.POST_HEADERS = {'X-Knack-REST-API-KEY':self.API_KEY,'X-Knack-Application-Id':self.APP_ID,'content-type':'application/json'}
        self.API_URL = f'https://api.knack.aft.org/v1/'
        self.LOADER_URL = f'https://loader.knack.aft.org/v1/applications/{self.APP_ID}'

        # INTERNAL #
        self.APP_DICT = {}

        res = requests.get(url=self.LOADER_URL)
        objects = res.json()['application']['objects']

        for obj in objects:
            fields = {}
            name = obj['name']
            key = obj['key']
            
            if name in TABLES:
                for item in obj['fields']:
                    fields.update({item['name'].lower():item['key']})
                self.APP_DICT.update({name.lower():{'obj_id':key,'fields':fields}})
        
    # function to return key for any value
    def get_key(self, dictionary ,val):
        for key, value in dictionary.items():
            if val == value:
                return key

        return ''
        
    
    # GET and format json from requestURL
    def getJSON(self, url):
        r = requests.get(url = self.API_URL + url, headers = self.GET_HEADERS)
        return r.json()
    
    def getObjectJSON(self, object_name):
        return (self.getJSON('objects/' + self.APP_DICT[object_name]['obj_id']))['object']
        
    def find_matches(self, object_name, field_name, match_val):
        field_id = self.APP_DICT[object_name]['fields'][field_name]
        object_id = self.APP_DICT[object_name]['obj_id']
        
        match_filter = {'match':'and', 'rules':[{'field':field_id, 'operator':'is', 'value': match_val}]}
        filter_for_url = urllib.parse.quote(json.dumps(match_filter))
        request_url = "objects/" + object_id + "/records?filters=" + filter_for_url
        res = self.getJSON(request_url)
        if res["total_records"] == 0:
            return ''
        else:
            return res["records"]
        
    def find_records_updated_at_date(self, knack_object, date):
        #Convert to IDs
        knack_object_id = self.APP_DICT[knack_object]['obj_id']
        field_to_match_id = self.APP_DICT[knack_object]['fields']['mdate']

        #Get Id
        match_filter = {'match':'and', 'rules':[{'field':field_to_match_id, 'operator':'is', 'value': date}]}
        filter_for_url = urllib.parse.quote(json.dumps(match_filter))
        request_url = "https://api.knack.aft.org/v1/objects/" + knack_object_id + "/records?filters=" + filter_for_url

        r = requests.get(url = request_url, headers = self.GET_HEADERS)
        #print(json.dumps(r.json(), indent=4))
        res_json_dict = json.loads(json.dumps(r.json()))
        if res_json_dict["total_records"] == 0:
            return ''
        else:
            return res_json_dict["records"]
        
    def get_userid(self, user):
        #Convert to IDs
        knack_object_id = self.APP_DICT['Accounts']['obj_id']
        knack_field_id = self.APP_DICT['Accounts']['fields']['KnackUserID']

        request_url = "https://api.knack.aft.org/v1/objects/" + knack_object_id + "/records/" +  user
        r = requests.get(url = request_url, headers = self.GET_HEADERS)
        #print(json.dumps(r.json(), indent=4))
        res_json_dict = json.loads(json.dumps(r.json()))
        return res_json_dict[knack_field_id]
    
    def get_connection(self,obj_name,field_name,connection):
        
        #Convert to IDs
        knack_object_id = self.APP_DICT[obj_name]['obj_id']
        knack_field_id = self.APP_DICT[obj_name]['fields'][field_name]

        #Get Id
        match_filter = {'match':'and', 'rules':[{'field':knack_field_id, 'operator':'is', 'value': connection}]}
        filter_for_url = urllib.parse.quote(json.dumps(match_filter))
        request_url = "https://api.knack.aft.org/v1/objects/" + knack_object_id + "/records?filters=" + filter_for_url
        r = requests.get(url = request_url, headers = self.GET_HEADERS)
        #print(json.dumps(r.json(), indent=4))
        res_json_dict = json.loads(json.dumps(r.json()))
        if res_json_dict["total_records"] == 0:
            return ''
        elif res_json_dict["total_records"] == 1:
            return res_json_dict["records"][0]["id"]
        else:
            return ''



def add_affiliate(record):
    aff_dict = client.APP_DICT['affiliate']['fields']
    connection_fields = ["parentaffiliateid","affiliatetypeid","affiliatedesignationid","affiliategeoreachid","affiliateinactivereasonid"]
    out = {}
    
    for k,v in record.items():
        if k in connection_fields:
            if k == 'affiliateinactivereasonid':
                conn = client.get_connection('affiliateinactivereason','affiliateinactivereasonid',v)
                out.update({aff_dict['affiliateinactivereason']:conn})

            elif k == 'parentaffiliateid':
                conn = client.get_connection('affiliate','affiliateid',v)
                out.update({aff_dict[k]:conn})
            else:
                conn = client.get_connection(k.replace('id',''),k,v)
                out.update({aff_dict[k]:conn})

        else:
            out.update({aff_dict[k]:v})

    out.update({aff_dict['aftdbupdate']:1})
    check_exists = client.get_connection('affiliate', 'affiliateid', record['affiliateid'])
    if check_exists:
        print(record['affiliateid'] + " exists! updating record...")
        request_url = "https://api.knack.aft.org/v1/objects/" + client.APP_DICT["affiliate"]['obj_id'] + "/records/" +  check_exists
        r = requests.put(url = request_url, headers = client.POST_HEADERS, data = json.dumps(out))
        print(r)
        if r.status_code != 200 and r.status_code != 500:
            print(json.dumps(r.json(), indent=4))
    else:
        print("Creating new affiliate with id: " + record['affiliateid'])
        request_url = "https://api.knack.aft.org/v1/objects/" + client.APP_DICT["affiliate"]['obj_id'] + "/records"
        r = requests.post(url = request_url, headers = client.POST_HEADERS, data = json.dumps(out))
        print(r)
        if r.status_code != 200 and r.status_code != 500:
            print(json.dumps(r.json(), indent=4))
        
        
def add_affiliatepercapita(record):
    record.pop('includestatepercapita')
    apc_dict = client.APP_DICT['affiliatepercapita']['fields']
    connection_fields = ['affiliateid', 'currentfiduciarybondcoverageid']
    out = {}
    
    for k,v in record.items():
        if k in connection_fields:
            if k == 'affiliateid':
                conn = client.get_connection('affiliate','affiliateid',v)
                out.update({apc_dict['affiliate']:conn})
            else:
                conn = client.get_connection('fiduciarybondcoverage','fiduciarybondcoverageid',v)
                out.update({apc_dict['fiduciarybondcoverage']:conn})

        else:
            out.update({apc_dict[k]:v})

    out.update({apc_dict['aftdbupdate']:1})
    check_exists = client.get_connection('affiliatepercapita', 'affiliatepercapitaid', record['affiliatepercapitaid'])
    if check_exists:
        print(record['affiliatepercapitaid'] + " exists! updating record...")
        request_url = "https://api.knack.aft.org/v1/objects/" + client.APP_DICT["affiliatepercapita"]['obj_id'] + "/records/" +  check_exists
        r = requests.put(url = request_url, headers = client.POST_HEADERS, data = json.dumps(out))
        print(r)
        if r.status_code != 200 and r.status_code != 500:
            print(json.dumps(r.json(), indent=4))
    else:
        print("Creating new affiliatepercapita with id: " + record['affiliatepercapitaid'])
        request_url = "https://api.knack.aft.org/v1/objects/" + client.APP_DICT["affiliatepercapita"]['obj_id'] + "/records"
        r = requests.post(url = request_url, headers = client.POST_HEADERS, data = json.dumps(out))
        print(r)
        if r.status_code != 200 and r.status_code != 500:
            print(json.dumps(r.json(), indent=4))
        
def add_affiliatedivision(record):
    adiv_dict = client.APP_DICT['affiliatedivision']['fields']
    connection_fields = ["affiliateid","divisionid"]
    out = {}
    
    for k,v in record.items():
        if k in connection_fields:
            if k == 'affiliateid':
                conn = client.get_connection('affiliate','affiliateid',v)
                out.update({adiv_dict['affiliate']:conn})
            elif k == 'divisionid':
                conn = client.get_connection('division','divisionid',v)
                out.update({adiv_dict['division']:conn})
        else:
            out.update({adiv_dict[k]:v})

    out.update({adiv_dict['aftdbupdate']:1})

    check_exists = client.get_connection('affiliatedivision', 'affiliatedivisionid', record['affiliatedivisionid'])
    if check_exists:
        print(record['affiliatedivisionid'] + " exists! updating record...")
        request_url = "https://api.knack.aft.org/v1/objects/" + client.APP_DICT["affiliatedivision"]['obj_id'] + "/records/" +  check_exists
        r = requests.put(url = request_url, headers = client.POST_HEADERS, data = json.dumps(out))
        print(r)
        if r.status_code != 200 and r.status_code != 500:
            print(json.dumps(r.json(), indent=4))
    else:
        print("Creating new affiliatedivision with id: " + record['affiliatedivisionid'])
        request_url = "https://api.knack.aft.org/v1/objects/" + client.APP_DICT["affiliatedivision"]['obj_id'] + "/records"
        r = requests.post(url = request_url, headers = client.POST_HEADERS, data = json.dumps(out))
        print(r)
        if r.status_code != 200 and r.status_code != 500:
            print(json.dumps(r.json(), indent=4))
            
            
def run_affiliate():
    table_name = os.environ['civis_table_path'] + 'sys_arts_affiliateuploader_xn'

    try:
        f = civis.io.read_civis(table=table_name,database="American Federation of Teachers",use_pandas=True)
    except civis.base.EmptyResultError as err:
        print('Empty upload table, aborting!...')
    else:
        f.fillna('', inplace=True)
        df = pd.DataFrame()
        df = f.astype(str)

        df['updatedat'] = pd.to_datetime(df.updatedat)
        df['updatedat'] = df['updatedat'].dt.strftime('%m/%d/%Y')
        
        df['createdat'] = pd.to_datetime(df.createdat)
        df['createdat'] = df['createdat'].dt.strftime('%m/%d/%Y')
        
        df['deletedat'] = pd.to_datetime(df.deletedat)
        df['deletedat'] = df['deletedat'].dt.strftime('%m/%d/%Y')

        for payload_dict in df.to_dict('records'):
            add_affiliate(payload_dict)
                
def run_affiliatepercapita():
    table_name = os.environ['civis_table_path'] + 'sys_arts_affiliatepercapitauploader_xn'

    try:
        f = civis.io.read_civis(table=table_name,database="American Federation of Teachers",use_pandas=True)
    except civis.base.EmptyResultError as err:
        print('Empty upload table, aborting!...')
    else:
        f.fillna('', inplace=True)
        df = pd.DataFrame()
        df = f.astype(str)
        
        df['updatedat'] = pd.to_datetime(df.updatedat)
        df['updatedat'] = df['updatedat'].dt.strftime('%m/%d/%Y')
        
        df['createdat'] = pd.to_datetime(df.createdat)
        df['createdat'] = df['createdat'].dt.strftime('%m/%d/%Y')
        
        df['deletedat'] = pd.to_datetime(df.deletedat)
        df['deletedat'] = df['deletedat'].dt.strftime('%m/%d/%Y')

        for payload_dict in df.to_dict('records'):
            add_affiliatepercapita(payload_dict)

def run_affiliatedivision():
    table_name = os.environ['civis_table_path'] + 'sys_arts_affiliatedivisionuploader_xn'

    try:
        f = civis.io.read_civis(table=table_name,database="American Federation of Teachers",use_pandas=True)
    except civis.base.EmptyResultError as err:
        print('Empty upload table, aborting!...')
    else:
        f.fillna('', inplace=True)
        df = pd.DataFrame()
        df = f.astype(str)
        
        df['updatedat'] = pd.to_datetime(df.updatedat)
        df['updatedat'] = df['updatedat'].dt.strftime('%m/%d/%Y')
        
        df['createdat'] = pd.to_datetime(df.createdat)
        df['createdat'] = df['createdat'].dt.strftime('%m/%d/%Y')
        
        df['deletedat'] = pd.to_datetime(df.deletedat)
        df['deletedat'] = df['deletedat'].dt.strftime('%m/%d/%Y')

    for payload_dict in df.to_dict('records'):
        add_affiliatedivision(payload_dict)

client = KnackAFT()


print("Uploading affiliatedivision...")
run_affiliatedivision()
print("--------------------------------")


Uploading affiliatedivision...
Creating new affiliatedivision with id: 999999
<Response [200]>
--------------------------------
